In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from PIL import Image

In [3]:
DATA_DIR = "./data/coco"

IMG_HEIGHT = 244
IMG_WIDTH = 244

SEED = 1

In [4]:
# Setup GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)


# Loading Data

In [5]:
# Load Image data
images = {}
image_dir = DATA_DIR + "/val2017/"

for filename in os.listdir(image_dir):
    if filename.endswith(".jpg"):
        img = tf.io.read_file(image_dir + filename)
        images[int(filename[:-4])] = tf.image.decode_jpeg(img, channels=3)

In [6]:
# Load bbox/annotations
with open(DATA_DIR + "/annotations/instances_val2017.json") as f:
    _ = json.load(f)
    categories = {entry["id"]:{
            "supercategory": entry["supercategory"],
            "name": entry["name"]
        } for entry in _["categories"]}
    instances = {entry["id"]:{
            "bbox": tuple(entry["bbox"]),
            "category_id": entry["category_id"],
            "image_id": entry["image_id"]
        } for entry in _["annotations"]}

# Env setup

In [7]:
from multiagent.infrastructure.env import ObjectLocalizationEnv, BetterRewardEnv, MovingEdgeEnv, HierarchicalZoomEnv, StretchyZoomEnv

In [8]:
model = tf.keras.models.Sequential(tf.keras.applications.VGG16(weights="imagenet", include_top = True).layers[:-2])

In [9]:
og_env = ObjectLocalizationEnv(model, (224, 224), feature_dim = 4096)
moving_edge_env = MovingEdgeEnv(model, (244, 244), feature_dim = 4096, transformation_factor = 0.4)
zoomzoom_env = HierarchicalZoomEnv(model, (244,244), feature_dim = 4096)
stretchyzoom_env = StretchyZoomEnv(model, (244,244), feature_dim = 4096)

# Training

In [10]:
from multiagent.infrastructure.trainer import RL_Trainer
from multiagent.agents.dqn_agent import DQN_Agent
from multiagent.util.dqn_utils import LinearSchedule

In [11]:
class EpochSchedule():
    def __init__(self, total_epochs, final_p, initial_p=1.0):
        self.epochs_til_final = total_epochs
        self.final_p = final_p
        self.initial_p = initial_p
        self.current_epoch = 0
        
    def value(self, t):
        fraction  = min(float(self.current_epoch) / self.epochs_til_final, 1.0)
        return self.initial_p + fraction * (self.final_p - self.initial_p)
    
    def increment_epoch(self):
        self.current_epoch += 1
        if self.current_epoch > self.epochs_til_final:
            self.current_epoch = self.epochs_til_final

In [12]:
agent_params = { #TODO!!!!
    "replay_buffer_size": 500,
    "batch_size": 32,
    "gamma": 0.1,
    "epsilon": EpochSchedule(15, final_p = .1, initial_p = 1),
    "dropout": 0.1
}

In [13]:
params = {
    "learning_freq": 1,
    "seed": SEED,
    "agent_class": DQN_Agent,
    "optimizer": tf.keras.optimizers.Adam(learning_rate = 0.001),
    "loss": tf.keras.losses.Huber(),
    "agent_params": agent_params,
    "env": None, #PlaceHolder
    "max_path_length": 200,
    "model_name": "austensux", #PlaceHolder
    "save_freq": 0 #Manually save after every epoch
}

In [14]:
#Find all images to train on

def find_all_instances(image_id, category_id):
    target_bboxs, keys = [], []
    for key, instance in instances.items():
        if instance['image_id'] == image_id and instance['category_id'] == category_id:
            target_bboxs.append(instance)
            keys.append(key)
    return keys, target_bboxs

def find_images_with_category(category_id):
    images = set()
    for key, instance in instances.items():
        if instance['category_id'] == category_id:
            images.add(instance['image_id'])
    return images

In [15]:
len(find_images_with_category(1))

2693

## Training Loop

In [16]:
from copy import deepcopy

In [17]:
used_agent_params = deepcopy(agent_params)
params['env'] = og_env
params['model_name'] = "caceido"
params['agent_params'] = used_agent_params

In [18]:
trainer = RL_Trainer(params=params)
category_trained = 1

In [21]:
category_images = find_images_with_category(category_trained)
for i in range(15):
    for image_id in images:
        keys, target_bboxs = find_all_instances(image_id, category_trained)
        target_bboxs = [bbox["bbox"] for bbox in find_all_instances(image_id, 1)[1]]
        image_tensor = tf.expand_dims(images[image_id], 0)
        params['env'].training_reset(target_bboxs, image_tensor)
        trainer.run_training_loop(1)
    used_agent_params['epsilon'].increment_epoch()
    trainer.agent.q_func.save("models/{}_iter{}".format(params['model_name'], i))



********** Iteration 0 ************


KeyboardInterrupt: 